In [1]:
# import sys
# import os
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [2]:
import numpy as np
from env import MufexKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import *
from quantfreedom.nb_funcs.nb_strategies.nb_strategy import *
from quantfreedom.nb_funcs.nb_simulate.nb_sim_base import nb_sim_backtest
from quantfreedom.helper_funcs import dl_ex_candles, candles_to_df

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [3]:
candles = mufex_main.get_candles(
    symbol="BTCUSDT",
    timeframe="5m",
    since_timestamp=1686787200000,
    until_timestamp=1687478400000,
)

In [4]:
candles_to_df(candles)

,timestamp,open,high,low,close,volume
datetime,,,,,,
2023-06-15 00:00:00,1686787200000,25115.0,25116.0,25079.5,25085.5,1.991
2023-06-15 00:05:00,1686787500000,25085.5,25102.5,25071.0,25095.5,1.578
2023-06-15 00:10:00,1686787800000,25095.5,25115.0,25095.5,25114.0,2.594
2023-06-15 00:15:00,1686788100000,25114.0,25135.5,25092.0,25130.0,2.197
2023-06-15 00:20:00,1686788400000,25130.0,25135.5,25126.0,25132.0,1.169
...,...,...,...,...,...,...
2023-06-22 23:35:00,1687476900000,29978.7,29988.6,29972.2,29972.8,1.925
2023-06-22 23:40:00,1687477200000,29972.8,29976.7,29935.9,29947.9,4.660
2023-06-22 23:45:00,1687477500000,29947.9,29969.3,29941.0,29946.6,6.870


In [5]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)

backtest_settings = BacktestSettings(
    array_size=1000,
    gains_pct_filter=0,
    total_trade_filter=40,
    qf_filter=0,
)

# backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([2, 4, 6, 8]) * 3,
    max_trades=np.array([5]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2, 4, 8, 10]),
    sl_based_on_add_pct=np.array([0.05, 0.1, 0.2]),
    sl_based_on_lookback=np.array([20, 40, 60]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.05, 0.1, 0.5, 1, 2]),
    trail_sl_when_pct=np.array([0.5, 1, 3]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=100,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

In [6]:
bt_strat_df = nb_sim_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    logger_bool=False,
    long_short="long",
    nb_strat_evaluate=nb_strat_long_evaluate,
    nb_strat_get_current_ind_settings=nb_strat_get_current_ind_settings,
    nb_strat_get_ind_set_str=nb_strat_get_ind_set_str,
    nb_strat_get_total_ind_settings=nb_strat_get_total_ind_settings,
    nb_strat_ind_creator=nb_strat_bt_create_ind,
    static_os=static_os,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 2,160
Total combinations of settings to test: 12,960
Total candles: 2,304
Total candles to test: 29,859,840


In [7]:
bt_strat_df.sort_values(by=["qf_score"], ascending=False)

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,total_pnl,ending_eq
261,4,2062,43.0,12,31,203.255,27.907,0.735,3192.821,2032.555,3032.555
360,5,2062,43.0,12,31,205.030,27.907,0.735,3202.702,2050.299,3050.299
23,2,982,43.0,12,31,147.448,27.907,0.733,2527.731,1474.475,2474.475
331,5,1522,43.0,12,31,187.219,27.907,0.732,3022.387,1872.188,2872.188
233,4,1522,43.0,12,31,187.018,27.907,0.731,3022.762,1870.183,2870.183
...,...,...,...,...,...,...,...,...,...,...,...
287,5,504,42.0,8,34,22.187,19.048,0.002,2056.699,221.865,1221.865
197,4,664,49.0,20,29,0.149,40.816,0.001,1222.687,1.485,1001.485
295,5,664,49.0,20,29,0.149,40.816,0.001,1222.687,1.485,1001.485
293,5,634,51.0,20,31,1.929,39.216,0.001,1415.570,19.286,1019.286
